In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.dml.color import RGBColor
import logging
import math

logger = logging.getLogger(__name__)

class AdvancedSlideManager:
    def __init__(self, template_path):
        self.prs = Presentation(template_path)
        self._validate_template()
        
        # Template metrics from user verification
        self.ROWS_PER_TEXTBOX = 16
        self.CHARS_PER_ROW = 31
        self.LINE_HEIGHT = Inches(5/16)  # 5" height / 16 rows
        
        # Layout tracking
        self.current_slide = None
        self.current_textbox = None
        self.slide_count = 0
        self.textbox_count = 0
        
        # Initialize first slide
        self._add_new_slide()

    def _validate_template(self):
        """Ensure template has required slides"""
        if len(self.prs.slides) < 1:
            raise ValueError("Template must contain at least one slide")

    def _add_new_slide(self):
        """Add new dual-column slide"""
        self.current_slide = self.prs.slides.add_slide(self.prs.slide_layouts[6])
        
        # Left column (4" width, 5" height)
        left = self.current_slide.shapes.add_textbox(Inches(0.5), Inches(1), Inches(4), Inches(5))
        left.name = "textMainBullets_L"
        left.text_frame.word_wrap = True
        left.text_frame.auto_size = MSO_AUTO_SIZE.NONE
        
        # Right column (4" width, 5" height)
        right = self.current_slide.shapes.add_textbox(Inches(4.7), Inches(1), Inches(4), Inches(5))
        right.name = "textMainBullets_R"
        right.text_frame.word_wrap = True
        right.text_frame.auto_size = MSO_AUTO_SIZE.NONE
        
        self.slide_count += 1
        self.current_textbox = left
        self.textbox_count = 0  # Reset column counter for new slide

    def _split_md_to_chunks(self, md_content):
        """Split markdown into textbox-sized chunks (16 rows x 31 chars)"""
        chunks = []
        current_chunk = []
        current_line = ""
        
        for line in md_content.split('\n'):
            line = line.strip()
            if not line:
                continue
                
            # Split line into 31-character segments
            words = line.split()
            for word in words:
                if len(current_line) + len(word) + 1 > self.CHARS_PER_ROW:
                    current_chunk.append(current_line.strip())
                    current_line = ""
                    if len(current_chunk) >= self.ROWS_PER_TEXTBOX:
                        chunks.append(current_chunk)
                        current_chunk = []
                current_line += f"{word} "
            
            if current_line:
                current_chunk.append(current_line.strip())
                current_line = ""
            
            if len(current_chunk) >= self.ROWS_PER_TEXTBOX:
                chunks.append(current_chunk)
                current_chunk = []
        
        if current_chunk:
            chunks.append(current_chunk)
        
        return chunks

    def _advance_container(self):
        """Move to next column or create new slide"""
        self.textbox_count += 1
        if self.textbox_count % 2 == 0:
            self._add_new_slide()
        else:
            self.current_textbox = self.current_slide.shapes[-1]  # Right column

    def _fill_textbox(self, chunk):
        """Fill current textbox with formatted content"""
        tf = self.current_textbox.text_frame
        tf.clear()
        
        for line in chunk:
            p = tf.add_paragraph()
            p.text = line
            p.font.size = Pt(14)
            
            # Header formatting
            if line.startswith('##'):
                p.font.bold = True
                p.font.size = Pt(18)
            elif line.startswith('###'):
                p.font.italic = True
                p.font.size = Pt(16)

    def process_md(self, md_content):
        """Process markdown with precise layout control"""
        chunks = self._split_md_to_chunks(md_content)
        logger.info(f"Content requires {len(chunks)} textboxes across {math.ceil(len(chunks)/2)} slides")
        
        for chunk in chunks:
            self._fill_textbox(chunk)
            self._advance_container()

    def save(self, output_path):
        """Save final presentation"""
        self.prs.save(output_path)
        logger.info(f"Saved presentation with {self.slide_count} slides")


# Usage example
if __name__ == "__main__":
    import sys
    if len(sys.argv) != 3:
        print("Usage: python report_generator.py input.md output.pptx")
        sys.exit(1)
    
    with open(sys.argv[1], 'r') as f:
        md_content = f.read()
    
    manager = AdvancedSlideManager("fixed_template.pptx")
    manager.process_md(md_content)
    manager.save(sys.argv[2])


In [14]:
def create_optimized_template():
    prs = Presentation()
    
    # Single-column slide (Title content)
    slide1 = prs.slides.add_slide(prs.slide_layouts[6])
    title_box = slide1.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1.5))
    title_box.name = "title_box"
    title_box.text_frame.word_wrap = False
    title_box.text_frame.auto_size = MSO_AUTO_SIZE.NONE

    # Dual-column slides (Main content)
    for _ in range(2):  # Create 2 master slides
        slide = prs.slides.add_slide(prs.slide_layouts[6])
        for idx, col in enumerate(['L', 'R']):
            left = Inches(0.5) if col == 'L' else Inches(4.7)
            textbox = slide.shapes.add_textbox(left, Inches(1.2), Inches(4), Inches(5))
            textbox.name = f"content_{col}"
            tf = textbox.text_frame
            tf.auto_size = MSO_AUTO_SIZE.NONE
            tf.word_wrap = True
            tf.vertical_anchor = MSO_VERTICAL_ANCHOR.TOP
            tf.margin_left = Inches(0.1)
            tf.margin_right = Inches(0.1)
    
    prs.save("fixed_template.pptx")


In [15]:
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts
Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations
Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies
Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities
Certificates of deposit (CDs)
Bankers' acceptances
Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase)
Building improvements capitalization
Machinery installation costs
Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings
Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income

"""

In [16]:
manager = AdvancedSlideManager("fixed_template.pptx")
manager.process_md(md_content)
manager.save("financial_report.pptx")

In [6]:
# another trial

from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.oxml.xmlchemy import OxmlElement
from dataclasses import dataclass
from typing import List
import re

@dataclass
class FinancialItem:
    accounting_type: str
    account_title: str
    descriptions: List[str]
    continued: bool = False

def parse_markdown(md_content: str) -> List[FinancialItem]:
    items = []
    current_type = ""
    current_title = ""
    current_descriptions = []
    
    for line in md_content.strip().split('\n'):
        line = line.strip()
        if not line:
            continue
            
        if line.startswith('## '):
            if current_type or current_title or current_descriptions:
                items.append(FinancialItem(
                    accounting_type=current_type,
                    account_title=current_title,
                    descriptions=current_descriptions
                ))
            current_type = line[3:].strip()
            current_title = ""
            current_descriptions = []
            
        elif line.startswith('### '):
            if current_title or current_descriptions:
                items.append(FinancialItem(
                    accounting_type=current_type,
                    account_title=current_title,
                    descriptions=current_descriptions
                ))
            current_title = line[4:].strip()
            current_descriptions = []
            
        else:
            current_descriptions.append(line)
    
    if current_type or current_title or current_descriptions:
        items.append(FinancialItem(
            accounting_type=current_type,
            account_title=current_title,
            descriptions=current_descriptions
        ))
    
    return items

def create_continued_header(text: str) -> str:
    return f"{text} (continued)"

def format_bullet(text: str, level: int) -> str:
    bullets = ['•', '◦', '-']
    return '  ' * level + bullets[level] + ' ' + text

def split_content(items: List[FinancialItem], max_lines=12) -> List[List[FinancialItem]]:
    """Improved content splitting with continuation markers"""
    chunks = []
    current_chunk = []
    current_lines = 0
    
    for item in items:
        lines_needed = sum([
            1 if item.accounting_type else 0,
            1 if item.account_title else 0,
            len(item.descriptions)
        ])
        
        if current_lines + lines_needed > max_lines:
            if current_chunk:
                # Mark continuation for last item
                current_chunk[-1].continued = True
                chunks.append(current_chunk)
                current_chunk = [create_continued_item(current_chunk[-1])]
                current_lines = lines_needed  # Reset with continued item
        else:
            current_chunk.append(item)
            current_lines += lines_needed
    
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

def create_continued_item(previous_item: FinancialItem) -> FinancialItem:
    """Create continuation item with proper inheritance"""
    return FinancialItem(
        accounting_type=previous_item.accounting_type,
        account_title=previous_item.account_title,
        descriptions=[],
        continued=True
    )

def apply_custom_formatting(paragraph, level: int):
    pPr = paragraph._p.get_or_add_pPr()
    pPr.set('marL', str(level * 360000))  # 0.5" per level
    pPr.set('indent', '0')
    
    if level == 0:
        paragraph.font.bold = True
        paragraph.font.size = Pt(14)
    elif level == 1:
        paragraph.font.size = Pt(12)
    else:
        paragraph.font.size = Pt(11)
        
def validate_template(prs):
    """Ensure template contains required placeholders"""
    if len(prs.slide_layouts) < 2:
        raise ValueError("Template requires at least 2 slide layouts")
    
    first_slide = prs.slides[0]
    if not any(shape.name == "textMainBullets" for shape in first_slide.shapes):
        raise ValueError("First slide missing 'textMainBullets' placeholder")

def get_layout(prs, page_num):
    """Get appropriate layout based on page number"""
    return prs.slide_layouts[1] if page_num >= 2 else prs.slide_layouts[0]

def get_section_placeholders(slide):
    """Retrieve left/right placeholders with improved validation"""
    placeholders = {
        shape.name: shape for shape in slide.shapes 
        if shape.is_placeholder and shape.name in ("textMainBullets_L", "textMainBullets_R")
    }
    
    if not placeholders:
        available = [f"'{shape.name}'" for shape in slide.shapes if shape.is_placeholder]
        raise ValueError(f"Required placeholders not found. Available: {', '.join(available)}")
    
    return (
        placeholders.get("textMainBullets_L"),
        placeholders.get("textMainBullets_R")
    )
    
def apply_continuation_marks(item):
    """Apply continuation marks to hierarchy levels"""
    return FinancialItem(
        accounting_type=f"{item.accounting_type} (cont'd)" if item.accounting_type else "",
        account_title=f"{item.account_title} (cont'd)" if item.account_title else "",
        descriptions=item.descriptions,
        continued=True
    )

def generate_pptx(template_path: str, items: List[FinancialItem]):
    prs = Presentation(template_path)
    
    # Validate template structure
    validate_template(prs)

    # Page 1 handling
    slide_1 = prs.slides[0]
    text_placeholder = next((shape for shape in slide_1.shapes 
                            if shape.is_placeholder and shape.name == "textMainBullets"), None)
    
    if text_placeholder:
        populate_placeholder(text_placeholder, items[:3])

    # Subsequent pages with layout validation
    content_chunks = split_content(items)
    for chunk_idx, chunk in enumerate(content_chunks[1:], start=2):
        try:
            slide = prs.slides.add_slide(get_layout(prs, chunk_idx))
            left_placeholder, right_placeholder = get_section_placeholders(slide)
            
            current_side = left_placeholder
            for item in chunk:
                if item.continued:
                    item = apply_continuation_marks(item)
                
                populate_placeholder(current_side, [item])
                current_side = right_placeholder if current_side == left_placeholder else left_placeholder
                
        except ValueError as e:
            print(f"Skipping page {chunk_idx} due to: {str(e)}")
            continue
    
    return prs

def populate_placeholder(placeholder, items):
    """Helper function to safely populate placeholder content"""
    if not placeholder.has_text_frame:
        raise ValueError("Placeholder does not support text")
    
    tf = placeholder.text_frame
    tf.clear()
    
    for item in items:
        if item.accounting_type:
            p = tf.add_paragraph()
            p.text = item.accounting_type
            p.level = 0
            p.font.bold = True
        
        if item.account_title:
            p = tf.add_paragraph()
            p.text = item.account_title
            p.level = 1
        
        for desc in item.descriptions:
            p = tf.add_paragraph()
            p.text = desc
            p.level = 2
            
# Usage example
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts
Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations
Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies
Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities
Certificates of deposit (CDs)
Bankers' acceptances
Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase)
Building improvements capitalization
Machinery installation costs
Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings
Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income
"""

# Ensure your template.pptx contains these named placeholders:
# - First slide: "textMainBullets"
# - Subsequent slides: "textMainBullets_L" and "textMainBullets_R"

parsed_items = parse_markdown(md_content)
presentation = generate_pptx("template.pptx", parsed_items)
presentation.save("financial_report.pptx")



Skipping page 2 due to: Required placeholders not found. Available: 'Title 1', 'Content Placeholder 2'
Skipping page 3 due to: Required placeholders not found. Available: 'Title 1', 'Content Placeholder 2'
Skipping page 4 due to: Required placeholders not found. Available: 'Title 1', 'Content Placeholder 2'
Skipping page 5 due to: Required placeholders not found. Available: 'Title 1', 'Content Placeholder 2'
Skipping page 6 due to: Required placeholders not found. Available: 'Title 1', 'Content Placeholder 2'
Skipping page 7 due to: Required placeholders not found. Available: 'Title 1', 'Content Placeholder 2'
Skipping page 8 due to: Required placeholders not found. Available: 'Title 1', 'Content Placeholder 2'
Skipping page 9 due to: Required placeholders not found. Available: 'Title 1', 'Content Placeholder 2'
Skipping page 10 due to: Required placeholders not found. Available: 'Title 1', 'Content Placeholder 2'
Skipping page 11 due to: Required placeholders not found. Available: 'Ti

In [9]:
from pptx import Presentation
from pptx.util import Inches
from dataclasses import dataclass
from typing import List
import re

@dataclass
class FinancialItem:
    accounting_type: str
    account_title: str
    descriptions: List[str]
    continued: bool = False

def parse_markdown(md_content: str) -> List[FinancialItem]:
    items = []
    current_type = ""
    current_title = ""
    current_descriptions = []
    
    for line in md_content.strip().split('\n'):
        line = line.strip()
        if not line:
            continue
            
        if line.startswith('## '):
            if current_type or current_title or current_descriptions:
                items.append(FinancialItem(current_type, current_title, current_descriptions))
            current_type = line[3:].strip()
            current_title = ""
            current_descriptions = []
        elif line.startswith('### '):
            if current_title or current_descriptions:
                items.append(FinancialItem(current_type, current_title, current_descriptions))
            current_title = line[4:].strip()
            current_descriptions = []
        else:
            current_descriptions.append(line)
    
    if current_type or current_title or current_descriptions:
        items.append(FinancialItem(current_type, current_title, current_descriptions))
    
    return items

def chunk_items(items: List[FinancialItem], items_per_slide=5) -> List[List[FinancialItem]]:
    chunks = []
    current_chunk = []
    
    for item in items:
        if len(current_chunk) >= items_per_slide:
            chunks.append(current_chunk)
            current_chunk = []
        current_chunk.append(item)
    
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

def add_slide_content(slide, items):
    content_placeholder = None
    for shape in slide.shapes:
        if shape.is_placeholder and shape.placeholder_format.idx == 1:
            content_placeholder = shape
            break
    
    if not content_placeholder:
        return
    
    text_frame = content_placeholder.text_frame
    text_frame.clear()
    
    for item in items:
        if item.accounting_type:
            p = text_frame.add_paragraph()
            p.text = item.accounting_type
            p.level = 0
            if item.continued:
                p.text += " (continued)"
        
        if item.account_title:
            p = text_frame.add_paragraph()
            p.text = item.account_title
            p.level = 1
            if item.continued:
                p.text += " (continued)"
        
        for desc in item.descriptions:
            p = text_frame.add_paragraph()
            p.text = desc
            p.level = 2

def generate_pptx(template_path: str, items: List[FinancialItem]):
    prs = Presentation(template_path)
    
    # Validate template structure
    if len(prs.slide_layouts) < 2:
        raise ValueError("Template requires at least 2 slide layouts (title + content)")

    # Page 1 - Use first layout (title slide)
    slide_1 = prs.slides.add_slide(prs.slide_layouts[0])
    title_placeholder = slide_1.shapes.title
    section_a_placeholder = next((shape for shape in slide_1.shapes if shape.name == "companyBackground"), None)
    
    # Populate company background (Section A)
    if section_a_placeholder:
        section_a_placeholder.text = "Company Name\nEst. 2020\nHQ: Hong Kong\nIndustry: Financial Services"

    # Page 1 Content (Sections B/C)
    text_placeholder = next((shape for shape in slide_1.shapes if shape.name == "textMainBullets"), None)
    if text_placeholder:
        populate_placeholder(text_placeholder, items[:3])

    # Subsequent pages using second layout (content slide)
    content_layout = prs.slide_layouts[1]
    for chunk in split_content(items[3:]):
        slide = prs.slides.add_slide(content_layout)
        
        # Get left/right placeholders from template
        left_ph = next((shape for shape in slide.shapes if shape.name == "textMainBullets_L"), None)
        right_ph = next((shape for shape in slide.shapes if shape.name == "textMainBullets_R"), None)
        
        # Distribute content according to template structure
        current_side = left_ph
        for item in chunk:
            if current_side:
                populate_placeholder(current_side, [item])
                current_side = right_ph if current_side == left_ph else left_ph

    return prs


# Usage
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts
Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations
Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies
Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities
Certificates of deposit (CDs)
Bankers' acceptances
Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase)
Building improvements capitalization
Machinery installation costs
Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings
Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income
"""

presentation = generate_pptx("template.pptx", md_content)
presentation.save("financial_report.pptx")


AttributeError: 'str' object has no attribute 'accounting_type'